**We use Pegasus financial summariser model from the transformers library in order to perform summarisation on the text obtained from 8K forms**

In [1]:
!pip install transformers
!pip install -q sentencepiece

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 29.2 MB/s 
     |████████████████████████████████| 596 kB 44.4 MB/s 
     |████████████████████████████████| 6.5 MB 33.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import json
import pandas as pd

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus").to(device)

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [5]:
def findSummary(sentence):
    batch = tokenizer(sentence, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text[0]

In [9]:
def findSummaries(json_file_name):
  f = open(f'/content/{json_file_name}')
  data = json.load(f)
  companies = pd.read_csv("/content/GoodCom.csv")
  ciks = companies["CIK"].astype(int).tolist()
  years = [2021,2020]
  summarised_8k = dict()
  for cik in ciks:
    year_dict = dict()
    for year in years:
      year_dict[year] = []
      try:
        sentences = data[f"{cik}"][f"{year} Sentences"]
        for sentence_object in sentences:
          sentence = sentence_object['sentence']
          date = sentence_object['date']

          summary = findSummary(sentence)
          summary = summary.capitalize()

          year_dict[year].append({'sentence':summary,'date':date})

      except Exception as e:
        print(e)
        continue
    print("cik ",cik," completed")
    summarised_8k[cik] = year_dict
  print("Done")
  with open(f"summarised_{json_file_name}.json", 'w') as f:
    json.dump(summarised_8k, f, indent=4)

In [ ]:
json_file_name = "8k_text_data.json" #name of json file obtained from bulk scraping

findSummaries(json_file_name)